## UAV fleet Simulation 

This is for testing propouses

In [1]:
# Custom modules
from config import Color
from helpers import clean#, local2global
from plan import Plan
from simulators import QGC, ConfigGazebo, Gazebo, Simulator
from helpers.change_coordinates import ENUs_to_GRAs
from mavlink.customtypes.location import ENUPose
from helpers.change_coordinates import rel_to_abs 
from mavlink.customtypes.location import GRAPose
clean()

In [ ]:
# 


# offsets= [  # east, north, up, heading
#     ENUPose(0., 0., 0., 0.),
#     ENUPose(10., 0., 0., 45.),
#     ENUPose(-5., -10., 0., 225.),
#     ENUPose(-15., 0., 0., 0.),
#     ENUPose(0., -20., 0., 0.),
# ]
# origin=GRAPose(-35.3633245, 149.1652241, 0, 0)


# n_vehicles = len(offsets)
# side_lens = (5, 5, 5, 5, 5)  # must agree with n_vehicles
# local_paths = [
#     Plan.create_square_path(side_len=side_len, alt=5, clockwise=True)
#     for side_len in side_lens
# ]

# GRAoffsets = ENUs_to_GRAs(origin, offsets)
# GRApaths = [
#     ENUs_to_GRAs(offset, loc_path)
#     for offset, loc_path in zip(GRAoffsets, local_paths)
# ]


In [ ]:
from pathlib import Path
from typing import Sequence
from dataclasses import dataclass

@dataclass
class GRAPose:
    lat: float
    lon: float
    alt: float
    heading: float


def save_waypoints_gcs(path: Path, poses: Sequence[GRAPose]) -> None:
    """
    Save a .waypoints file from a sequence of GRAPose objects.
    The file will include:
    - Home (index 0, altitude = 0)
    - Takeoff (index 1, to the first pose with alt)
    - Mission waypoints (indices 2 to N)
    - Return to launch (last index, with alt = 0)
    """
    with path.open("w") as f:
        f.write("QGC WPL 110\n")

        # Home location
        home = poses[0]
        f.write(f"0\t1\t0\t16\t0\t0\t0\t0\t{home.lat:.7f}\t{home.lon:.7f}\t0.0\t1\n")

        # Takeoff at first pose altitude
        f.write(f"1\t0\t3\t22\t0\t0\t0\t0\t{home.lat:.7f}\t{home.lon:.7f}\t{home.alt:.1f}\t1\n")

        # Mission waypoints
        for i, pose in enumerate(poses[1:], start=2):
            f.write(f"{i}\t0\t3\t16\t0\t0\t0\t0\t{pose.lat:.7f}\t{pose.lon:.7f}\t{pose.alt:.1f}\t1\n")

        # Return to Launch (RTL)
        last = poses[-1]
        rtl_index = len(poses) + 1
        f.write(f"{rtl_index}\t0\t3\t21\t0\t0\t0\t0\t{last.lat:.7f}\t{last.lon:.7f}\t0.0\t1\n")



In [8]:
save_waypoints_gcs(Path("plan/missions/square_mission.waypoints"),GRApaths[0])



In [3]:
[Plan.create_square_path(side_len=side_len, alt=5) for side_len in side_lens]

[[ENU(x=0, y=0, z=5),
  ENU(x=0, y=5, z=5),
  ENU(x=5, y=5, z=5),
  ENU(x=5, y=0, z=5),
  ENU(x=0, y=0, z=5)],
 [ENU(x=0, y=0, z=5),
  ENU(x=0, y=5, z=5),
  ENU(x=5, y=5, z=5),
  ENU(x=5, y=0, z=5),
  ENU(x=0, y=0, z=5)],
 [ENU(x=0, y=0, z=5),
  ENU(x=0, y=5, z=5),
  ENU(x=5, y=5, z=5),
  ENU(x=5, y=0, z=5),
  ENU(x=0, y=0, z=5)],
 [ENU(x=0, y=0, z=5),
  ENU(x=0, y=5, z=5),
  ENU(x=5, y=5, z=5),
  ENU(x=5, y=0, z=5),
  ENU(x=0, y=0, z=5)],
 [ENU(x=0, y=0, z=5),
  ENU(x=0, y=5, z=5),
  ENU(x=5, y=5, z=5),
  ENU(x=5, y=0, z=5),
  ENU(x=0, y=0, z=5)]]

## Create Plans

In [4]:
offset = (0,0,0,0)  # east, north, up, heading 
origin=(-35.3633245, 149.1652241, 0, 0)
local_path = Plan.create_square_path(side_len=5, alt=5)
#plan = Plan.basic(wps=local_path, wp_margin=0.5)
plan = Plan.auto(mission_name="simple_mission") 

In [ ]:
from helpers.change_coordinates import ENU_to_GRA
homes = ENU_to_GRA(origin, [(0,0,0,0),(0,5,0,0),(5,0,0,0)]
                   )
global_paths = [] 
= enu_spawns_to_global_rel_alt(origin, [(0,0,0,0),(0,5,0,0),(5,0,0,0)])

In [ ]:
home = offset[:3]
global_path = local2global(local_path, home)

In [ ]:
home

(0, 0, 0)

In [ ]:
global_path

array([[0, 0, 5],
       [0, 5, 5],
       [5, 5, 5],
       [5, 0, 5],
       [0, 0, 5]])

## Visualization Parameters

In [ ]:
home = offset[:3]
global_path = local2global(local_path, home)
color = Color.BLUE
markers = [ConfigGazebo.create_trajectory_from_array(array=global_path,color=color)]
# Gazebo.plot_3d_interactive(markers=markers,title="Simulation Waypoints",frames=(0.2, 0.2, 0.6),ground=-0.05)

## Choose Simulator

In [ ]:
# simulator = Simulator(offsets=[offset], plans=[plan])

simulator = QGC(offsets=[offset], plans=[plan], origin=(-35.3633245, 149.1652241, 0, 0))


# gazebo_config = ConfigGazebo(world_path="simulators/gazebo/worlds/runway.world",
#                            models=[("iris",color)],
#                            markers = markers)
# simulator = Gazebo(offsets=[offset],plans=[plan],config=gazebo_config)


Launch Simulator

In [ ]:
orac = simulator.launch(gcs_sysids={'blue 🟦':[1]})

🚀 ArduPilot SITL vehicle 1 launched (PID 1959739)
🚀 UAV logic for vehicle 1 launched (PID 1959740)
🚀 Proxy for vehicle 1 launched (PID 1959741)
🔗 UAV logic 1 is connected to Ardupilot SITL vehicle 1
System id: 1
[Errno 111] Connection refused sleeping
System id: 1
SIM_VEHICLE: Start
SIM_VEHICLE: Killing tasks
SIM_VEHICLE: Starting up at [-35.3633245, 149.1652241, 0.0, 0.0]
SIM_VEHICLE: Using defaults from (../../ardupilot/Tools/autotest/default_params/copter.parm)
SIM_VEHICLE: Adding parameters from (/home/abeldg/uav-cyber-sim/params/vehicle.parm)
Setting SIM_SPEEDUP=1.000000
Setting MAV_SYSID=1
Suggested EK3_DRAG_BCOEF_* = 17.209, EK3_DRAG_MCOEF = 0.209
Home: -35.363324 149.165224 alt=0.000000m hdg=0.000000
Starting sketch 'ArduCopter'
Starting SITL input
Using Irlock at port : 9005
Waiting for connection ....


bind port 5770 for SERIAL0
SERIAL0 on TCP port 5770


Connection on serial port 5770

🚀 Starting Proxy 1
Loaded defaults from ../../ardupilot/Tools/autotest/default_params/copter.parm,/home/abeldg/uav-cyber-sim/params/vehicle.parm
Smoothing reset at 0.001
⬅️ GCS ← ARP 1: BAD_DATA
⬅️ ORC ← ARP 1: BAD_DATA
⬅️ VEH ← ARP 1: BAD_DATA
⬅️ GCS ← ARP 1: BAD_DATA
⬅️ ORC ← ARP 1: BAD_DATA
⬅️ VEH ← ARP 1: BAD_DATA
⬅️ GCS ← ARP 1: BAD_DATA
⬅️ ORC ← ARP 1: BAD_DATA
⬅️ VEH ← ARP 1: BAD_DATA
⬅️ GCS ← ARP 1: BAD_DATA
⬅️ ORC ← ARP 1: BAD_DATA
⬅️ VEH ← ARP 1: BAD_DATA
⬅️ GCS ← ARP 1: BAD_DATA
⬅️ ORC ← ARP 1: BAD_DATA
⬅️ VEH ← ARP 1: BAD_DATA
⬅️ GCS ← ARP 1: BAD_DATA
⬅️ ORC ← ARP 1: BAD_DATA
⬅️ VEH ← ARP 1: BAD_DATA
⬅️ GCS ← ARP 1: BAD_DATA
⬅️ ORC ← ARP 1: BAD_DATA
⬅️ VEH ← ARP 1: BAD_DATA
⬅️ GCS ← ARP 1: BAD_DATA
⬅️ ORC ← ARP 1: BAD_DATA
⬅️ VEH ← ARP 1: BAD_DATA
⬅️ GCS ← ARP 1: BAD_DATA
⬅️ ORC ← ARP 1: BAD_DATA
⬅️ VEH ← ARP 1: BAD_DATA
⬅️ GCS ← ARP 1: BAD_DATA
⬅️ ORC ← ARP 1: BAD_DATA
⬅️ VEH ← ARP 1: BAD_DATA
⬅️ GCS ← ARP 1: BAD_DATA
⬅️ ORC ← ARP 1: BAD_DAT

bind port 5772 for SERIAL1
SERIAL1 on TCP port 5772
bind port 5773 for SERIAL2
SERIAL2 on TCP port 5773
validate_structures:528: Validating structures
Waiting for internal clock bits to be set (current=0x00)


⬅️ GCS ← ARP 1: HEARTBEAT
⬅️ ORC ← ARP 1: HEARTBEAT
⬅️ VEH ← ARP 1: HEARTBEAT
🔗 UAV logic 1 is connected to Oracle ⚪
✅ Heartbeat received

🚀 Starting Vehicle 1 logic
Vehicle 1: ▶️ Plan Started: 📋 
➡️ VEH → ARP 1: HEARTBEAT
🚀 GCS blue 🟦 launched (PID 1959916)
🗺️ QGroundControl launched for 2D visualization — simulation powered by ArduPilot SITL.


## Execute Plan

In [ ]:
while len(orac.conns):
    for sysid in list(orac.conns.keys()):
        if orac.is_plan_done(sysid):
            orac.remove(sysid)

Vehicle 1: ▶️ Action Started: 📤🚁  UPLOAD_MISSION
⬅️ GCS ← ARP 1: HEARTBEAT
⬅️ ORC ← ARP 1: HEARTBEAT
⬅️ VEH ← ARP 1: HEARTBEAT
Vehicle 1: ▶️ Step Started: clear uav missions
➡️ VEH → ARP 1: MISSION_CLEAR_ALL
⬅️ GCS ← ARP 1: MISSION_ACK
⬅️ ORC ← ARP 1: MISSION_ACK
⬅️ VEH ← ARP 1: MISSION_ACK
🎉 Mission upload successful!
Vehicle 1: ✅ Step Done: clear uav missions
✅ 4 waypoints read.
🧭 Mission[0] → cmd: WAYPOINT, x: -35.3633245, y: 149.1652241, z: 0.0, current: 0
🧭 Mission[1] → cmd: TAKEOFF, x: -35.3633245, y: 149.1652241, z: 5.0, current: 0
🧭 Mission[2] → cmd: WAYPOINT, x: -35.3632795, y: 149.1652241, z: 5.0, current: 0
🧭 Mission[3] → cmd: LAND, x: -35.3632795, y: 149.1652241, z: 0.0, current: 0
⬅️ GCS ← ARP 1: STATUSTEXT
⬅️ ORC ← ARP 1: STATUSTEXT
⬅️ VEH ← ARP 1: STATUSTEXT
⬅️ GCS ← ARP 1: STATUSTEXT
⬅️ ORC ← ARP 1: STATUSTEXT
⬅️ VEH ← ARP 1: STATUSTEXT
⬅️ GCS ← ARP 1: BAD_DATA
⬅️ ORC ← ARP 1: BAD_DATA
⬅️ VEH ← ARP 1: BAD_DATA
⬅️ GCS ← ARP 1: BAD_DATA
⬅️ ORC ← ARP 1: BAD_DATA
⬅️ VEH ← A

📤 GCS ← UAV 1: Sending DONE (attempt 2)
📤 GCS ← UAV 1: Sending DONE (attempt 3)
📤 GCS ← UAV 1: Sending DONE (attempt 4)
📤 GCS ← UAV 1: Sending DONE (attempt 5)
📤 GCS ← UAV 1: Sending DONE (attempt 6)
📤 GCS ← UAV 1: Sending DONE (attempt 7)
📤 GCS ← UAV 1: Sending DONE (attempt 8)
⬅️ GCS ← ARP 1: HEARTBEAT
⬅️ ORC ← ARP 1: HEARTBEAT
⬅️ VEH ← ARP 1: HEARTBEAT
📤 GCS ← UAV 1: Sending DONE (attempt 9)
📤 GCS ← UAV 1: Sending DONE (attempt 10)
📤 GCS ← UAV 1: Sending DONE (attempt 11)
📤 GCS ← UAV 1: Sending DONE (attempt 12)
📤 GCS ← UAV 1: Sending DONE (attempt 13)
📤 GCS ← UAV 1: Sending DONE (attempt 14)
✅ Vehicle 1 completed its mission
✅ All UAVs assigned to GCS blue 🟦 have completed their mission.
➡️ GCS → ARP 1: COMMAND_ACK
💾 Trajectories saved to 'trajectories_blue 🟦.pkl'.
✅ ACK received. DONE message acknowledged.
❎ Vehicle 1 logic stopped.Connection reset or closed by peer on TCP socket

❎ Proxy 1 stopped.
Closed connection on SERIAL0
Closed connection on SERIAL1
